<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Hello, world!')
print('change')

Hello, world!
change
